<a href="https://colab.research.google.com/github/bmox/image-compression/blob/main/image_compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I hear about this api name from [this](https://youtu.be/mTZ5loRYVgQ) channel. Then I create my version.<br>
First visit https://tinypng.com/developers . Then give a name and email address to get the api key. Only first 500 images are free. So if you think your data is important, use your real email address with only 500 images. But if you thinking your data is not important then use any disposable mail like https://temp-mail.org/ and with every mail you will get 500 images upload option.Then copy the tinify api key.


In [260]:
# @title ##**Install required modules** { display-mode: "form" }
from IPython.display import clear_output
!pip3 install tinify
!pip3 install --upgrade tinify
!pip3 install kora
clear_output()

In [280]:
# @title ##**Enter your api key** { display-mode: "form" }

Enter_tinify_api_key= "qcs5R3BgP8hgH9jXwkpzm2fwyXnZgDqv" #@param {type:"string"}

try:
  tinify.key = Enter_tinify_api_key
  tinify.validate()
  print("Authentication Successful")
except Exception as e:
  print("Authentication failed!")
  print(str(e))

Authentication Successful


In [285]:
# @title ##**Select storage type** { display-mode: "form" }

from google.colab import drive
from google.colab import output
from google.colab import files
from google.colab.output import eval_js
from IPython.display import display, Javascript
from base64 import b64decode
import requests
import math
import os
import glob
import tinify
storage="google drive"#@param["temporary","google drive"]
if storage=="google drive":
  drive.mount('/content/gdrive')
  clear_output()
  print("Using drive as a storage")
elif storage=="temporary":
  os.chdir("/content/")
  print("Using colab as a storage")


if storage=="temporary":  
      os.chdir("/content/")
      try:
        os.mkdir("tinify")
        os.chdir("/content/tinify")
        try:
          os.mkdir("input")
          os.mkdir("output")
        except:
          pass
      except:
        os.chdir("/content/tinify")
        try:
          os.mkdir("input")
          os.mkdir("output")
        except:
          pass

elif storage=="google drive":     
  os.chdir("/content/gdrive/MyDrive")
  try:
    os.mkdir("tinify")
    os.chdir("/content/gdrive/MyDrive/tinify")
    try:
      os.mkdir("input")
      os.mkdir("output")
    except:
       pass
  except:
    os.chdir("/content/gdrive/MyDrive/tinify")
    try:
      os.mkdir("input")
      os.mkdir("output")
    except:
      pass


#success sound notification
def successful():
  output.eval_js('new Audio("https://www.myinstants.com/media/sounds/anime-wow-sound-effect.mp3").play()')    

#error sound notification
def error():
  output.eval_js('new Audio("https://www.myinstants.com/media/sounds/movie_1.mp3").play()')  

#get file size
def convert_size(size_bytes):
   if size_bytes == 0:
       return "0B"
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i]) 
#compress file size
def compress(input_file_path,output_file_path):
  source = tinify.from_file(input_file_path)
  source.to_file(output_file_path)
  size1= os.path.getsize(input_file_path)
  size2= os.path.getsize(output_file_path)
  try:
    print("Before compressing:")
    print(input_file_path.split("/")[-1]+" : "+ convert_size(size1))
    print("After compressing:")
    print(output_file_path.split("/")[-1]+" : "+convert_size(size2))
  except:
    pass


#download from google drive
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

#capture image
def take_photo(filename='input_capture.jpg', quality=1):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename   

#get image name
def get_image_name():
  image_type=[".jpg",".jpeg",".png",".PNG"]
  image_names=[]
  for i in image_type:
    path=os.getcwd()+f"/*{i}"
    for j in glob.glob(path):
      image_names.append(j)
  return image_names
print(f"We are using this folder: {os.getcwd()}") 
parent_path=os.getcwd()

Using drive as a storage
We are using this folder: /content/gdrive/MyDrive/tinify


<br>
<br>
Run this for image.
<br>
<br>

In [197]:
# @title ##**Image Upload type** { display-mode: "form" }



download_file_in_your_device=""
parent_path=os.getcwd()
# print(parent_path)
os.chdir(parent_path+"/input/")
for i in os.listdir():
  os.remove(i)


upload_type="Download a Image" #@param["Upload a image","Download a Image","Paste_path","Take a image"]
if upload_type=="Upload a image":
  uploaded = files.upload()
  clear_output()
  if not get_image_name():
    print("Upload canceled !")  
  else:
    before_rename=get_image_name()[0].split("/")[-1]
    input_rename="input_"+before_rename
    output_rename="output_"+before_rename
    os.rename(before_rename,input_rename)  
    
    compress(get_image_name()[0],parent_path+"/output/"+output_rename)
    # print(get_image_name())  
    download_file_in_your_device=parent_path+"/output/"+output_rename


elif upload_type=="Download a Image":
  download_link=input("Enter the download link:")
  clear_output()
  if "drive.google.com" in  download_link:
    file_id = download_link.split('/')[-2]
    give_file_name ="input.jpg"
    try:
      download_file_from_google_drive(file_id, give_file_name)
      if not get_image_name():
        print("Upload corrupted !")
      else:
        print("Download successful!")
        compress(get_image_name()[0],parent_path+"/output/"+"output.jpg")
        download_file_in_your_device=parent_path+"/output/"+"output.jpg"


    except:
      print("Download failed try again")     
  else: 
    try:
      source = tinify.from_url(download_link)
      source.to_file(parent_path+"/output/optimized.jpg")
      download_file_size=os.path.getsize(parent_path+"/output/optimized.jpg")
      download_file_in_your_device=parent_path+"/output/optimized.jpg"
      print("After compressing the file size:")
      print(f"optimized.jpg : {convert_size(download_file_size)}")
    except:
      print("failed")
elif upload_type=="Paste_path":
  paste_path=input("Enter image full_path")
  clear_output()
  compress(paste_path,parent_path+"/output/"+"output_"+paste_path.split("/")[-1])
  download_file_in_your_device=parent_path+"/output/"+"output_"+paste_path.split("/")[-1]

#take a image
elif upload_type=="Take a image":
  try:
    filename = take_photo()
    print("Image captured successfully.")
    # print('Saved to {}'.format(filename))
    # display(Image(filename))
    compress(get_image_name()[0],parent_path+"/output/"+"output_capture.jpg")
    download_file_in_your_device=parent_path+"/output/"+"output_capture.jpg"
  except Exception as err:
    print(str(err))

os.chdir(parent_path)    

After compressing the file size:
optimized.jpg : 62.23 KB


In [198]:
# @title ##**Download your compressed image** { display-mode: "form" }

files.download(download_file_in_your_device)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<br>
<br>
Run this for bulk images.
<br>
<br>

In [286]:
# @title ##**For bluk images** { display-mode: "form" }
#@markdown ##### the folder must be"Anyone with the link" option

#remove useless folder
directories=[name for name in os.listdir(".") if os.path.isdir(name)]
directories.remove("input")
directories.remove("output")
try:
  bluck_folder_name=str(directories[0])
except:
  pass  
os.system("rm -rf "+parent_path+"/"+bluck_folder_name) 


#download the folder

# past_drive_folder_link= "https://drive.google.com/drive/folders/1FOXyEENNGsEPJ7eJ0b-LHuD7FUdm9Srw?usp=sharing" #@param {type:"string"}

# folder_id= past_drive_folder_link.split('/')[-1].split("?")[0]
drive_folder_id= "1FOXyEENNGsEPJ7eJ0b-LHuD7FUdm9Srw" #@param {type:"string"}

from kora import drive
drive.download_folder(drive_folder_id)
clear_output()



os.chdir(parent_path)
directories=[name for name in os.listdir(".") if os.path.isdir(name)]
directories.remove("input")
directories.remove("output")
bluck_folder_name=str(directories[0])
os.chdir(parent_path+"/"+bluck_folder_name)
image_list=get_image_name()
os.chdir(parent_path)
os.chdir(parent_path+"/output")
try:
  os.mkdir(bluck_folder_name)
except:
  pass
os.chdir(parent_path)
for i in image_list:
  output_file_name=i.split("/")[-1]
  compress(i,parent_path+"/output/"+bluck_folder_name+"/"+output_file_name)
clear_output()

print("The compressed bluck folder path:")
print(parent_path+"/output/"+bluck_folder_name)  
os.chdir(parent_path)

The compressed bluck folder path:
/content/gdrive/MyDrive/tinify/output/test2


In [287]:
# @title ##**Download your bluk images zip file** { display-mode: "form" }
zip_path=parent_path+"/output/"+bluck_folder_name
os.system(f"zip -r {bluck_folder_name}.zip "+zip_path)
files.download(parent_path+"/"+bluck_folder_name+".zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>